In [ ]:
import os
import yfinance as yf

ticker = "^GSPC"
start_date = "1989-12-01"  # Start earlier to get Dec 1989
end_date = "2025-12-31"

spx = yf.download(ticker, start=start_date, end=end_date, progress=False)
spx.columns = spx.columns.get_level_values(0)
spx = spx[['Close']].rename(columns={'Close': 'ClosePrice'})

spx['DailyReturn'] = spx['ClosePrice'].pct_change()
spx['RV'] = spx['DailyReturn'] ** 2

spx['Month'] = spx.index.to_period('M')
monthly = spx['ClosePrice'].resample('ME').last().pct_change()
monthly.index = monthly.index.to_period('M')
spx['MonthlyReturn'] = spx['Month'].map(monthly)

vix = yf.download("^VIX", start=start_date, end=end_date, progress=False)
vix.columns = vix.columns.get_level_values(0)
vix = vix[['Close']].rename(columns={'Close': 'VIX'})
spx = spx.merge(vix, left_index=True, right_index=True, how='left')

spx = spx.dropna(subset=['VIX'])

spx = spx.reset_index()
print(spx.head())
print(f"MonthlyReturn NaN count: {spx['MonthlyReturn'].isna().sum()}")

os.makedirs("data/raw", exist_ok=True)
spx.to_csv("data/raw/sp500_raw.csv", index=False)
print("Saved: data/raw/sp500_raw.csv")

Price       Date  ClosePrice  DailyReturn        RV    Month  MonthlyReturn  \
0     1990-01-02  359.690002     0.017799  0.000317  1990-01      -0.068817   
1     1990-01-03  358.760010    -0.002586  0.000007  1990-01      -0.068817   
2     1990-01-04  355.670013    -0.008613  0.000074  1990-01      -0.068817   
3     1990-01-05  352.200012    -0.009756  0.000095  1990-01      -0.068817   
4     1990-01-08  353.790009     0.004514  0.000020  1990-01      -0.068817   

Price        VIX  
0      17.240000  
1      18.190001  
2      19.219999  
3      20.110001  
4      20.260000  
MonthlyReturn NaN count: 0
Saved: data/raw/sp500_raw.csv
